In [1]:
from snn_delays.snn import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils import train, get_device, propagate_batch, to_plot, set_seed
from snn_delays.utils.test_behavior import tb_addtask
from snn_delays.utils.visualization_utils import plot_taus
device = get_device()

c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on: cuda:0


In order to have unique batches in "episodic" sampling or procedural sampling, we must disable caching!

In [2]:
#set_seed(42)

time_window = 50
batch_size = 128 # 128: anil kag

ckpt_dir = 'sequential_tests_24_2_25'

dataset = 'addtask_episodic'
#dataset = 'addtask' 

DL = DatasetLoader(dataset=dataset, caching='', num_workers=0, batch_size=batch_size, total_time=time_window)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()
dataset_dict["time_ms"] = 2e3
#dataset_dict["time_ms"] = 50

In [3]:
structure = (10, 2)
snn_mf = SNN(dataset_dict, structure=structure, connection_type='mf', delay=None, delay_type='',
           reset_to_zero=False, win=time_window, loss_fn='mem_prediction', batch_size=batch_size, device=device, debug=True)

snn_mf.multi_proj = 50

snn_mf.set_network()
snn_mf.use_amp = False
snn_mf.model_name = f'addask_d_50_50'
snn_mf.input2spike_th = None
snn_mf.num_train_samples = batch_size


[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
2000.0
Delta t: 40.0 ms
mean of normal: -1.8545865421311407


In [4]:
train(snn_mf, train_loader, test_loader, 1e-3, 10, ckpt_dir=ckpt_dir, test_behavior=tb_addtask, scheduler=(100, 0.95), freeze_taus=True)

training addask_d_50_50 for 10 epochs...
Epoch [1/10], learning_rates 0.001000, 0.100000
tensor([[1.1900, 1.1900, 1.1900,  ..., 1.1900, 1.1900, 1.1900],
        [1.4651, 1.4651, 1.4651,  ..., 1.4651, 1.4651, 1.4651],
        [0.5976, 0.5976, 0.5976,  ..., 0.5976, 0.5976, 0.5976],
        ...,
        [0.9045, 0.9045, 0.9045,  ..., 0.9045, 0.9045, 0.9045],
        [0.8546, 0.8546, 0.8546,  ..., 0.8546, 0.8546, 0.8546],
        [1.2614, 1.2614, 1.2614,  ..., 1.2614, 1.2614, 1.2614]],
       device='cuda:0')
Step [1/1], Loss: 0.75193
Time elasped: 0.37712907791137695
Epoch [2/10], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([128, 10])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


tensor([[1.1978, 1.1978, 1.1978,  ..., 1.1978, 1.1978, 1.1978],
        [0.7947, 0.7947, 0.7947,  ..., 0.7947, 0.7947, 0.7947],
        [1.6192, 1.6192, 1.6192,  ..., 1.6192, 1.6192, 1.6192],
        ...,
        [0.6750, 0.6750, 0.6750,  ..., 0.6750, 0.6750, 0.6750],
        [1.5603, 1.5603, 1.5603,  ..., 1.5603, 1.5603, 1.5603],
        [1.2609, 1.2609, 1.2609,  ..., 1.2609, 1.2609, 1.2609]],
       device='cuda:0')
Step [1/1], Loss: 0.77050
Time elasped: 0.18027019500732422
Epoch [3/10], learning_rates 0.001000, 0.100000
tensor([[1.5743, 1.5743, 1.5743,  ..., 1.5743, 1.5743, 1.5743],
        [0.8114, 0.8114, 0.8114,  ..., 0.8114, 0.8114, 0.8114],
        [0.7592, 0.7592, 0.7592,  ..., 0.7592, 0.7592, 0.7592],
        ...,
        [0.3869, 0.3869, 0.3869,  ..., 0.3869, 0.3869, 0.3869],
        [0.6577, 0.6577, 0.6577,  ..., 0.6577, 0.6577, 0.6577],
        [1.1904, 1.1904, 1.1904,  ..., 1.1904, 1.1904, 1.1904]],
       device='cuda:0')
Step [1/1], Loss: 0.51145
Time elasped: 0.175434

In [6]:
refs = []
for x in range(200):
        _, labels = propagate_batch(snn_mf, train_loader)
        ref = labels[:,5:,0].T.cpu().numpy()
        out = snn_mf.mem_state['output'][int(0.9*time_window):].detach().cpu().numpy()

        #print(labels)
        print(ref[0][0])
        print('--------------')
        refs.append(ref[0][0])
        #print(out)

1.7136078
--------------
1.5091248
--------------
1.021637
--------------
1.4748383
--------------
1.2240906
--------------
1.5293121
--------------
1.0679016
--------------
1.3389587
--------------
1.1376495
--------------
1.0477448
--------------
1.5675507
--------------
0.06889343
--------------
0.70092773
--------------
0.9018402
--------------
0.8581848
--------------
0.20864868
--------------
1.5592041
--------------
0.21469116
--------------
0.44506836
--------------
0.33380127
--------------
0.708786
--------------
0.36499023
--------------
1.3300629
--------------
0.87161255
--------------
0.23083496
--------------
0.7825012
--------------
1.4906464
--------------
1.4910431
--------------
1.0254669
--------------
1.6647186
--------------
1.4189453
--------------
0.90501404
--------------
0.30986023
--------------
1.3889923
--------------
0.87342834
--------------
1.0052185
--------------
0.6659088
--------------
0.6404114
--------------
1.1413574
--------------
1.1138306
-----

In [7]:
refs = [float(ref) for ref in refs]

In [8]:
refs

[1.7136077880859375,
 1.509124755859375,
 1.021636962890625,
 1.4748382568359375,
 1.224090576171875,
 1.5293121337890625,
 1.067901611328125,
 1.338958740234375,
 1.1376495361328125,
 1.0477447509765625,
 1.5675506591796875,
 0.0688934326171875,
 0.700927734375,
 0.9018402099609375,
 0.858184814453125,
 0.208648681640625,
 1.5592041015625,
 0.214691162109375,
 0.445068359375,
 0.33380126953125,
 0.7087860107421875,
 0.364990234375,
 1.3300628662109375,
 0.871612548828125,
 0.2308349609375,
 0.782501220703125,
 1.4906463623046875,
 1.4910430908203125,
 1.0254669189453125,
 1.6647186279296875,
 1.4189453125,
 0.9050140380859375,
 0.3098602294921875,
 1.3889923095703125,
 0.8734283447265625,
 1.005218505859375,
 0.6659088134765625,
 0.640411376953125,
 1.141357421875,
 1.11383056640625,
 1.229949951171875,
 1.4955902099609375,
 0.9365234375,
 1.0824127197265625,
 1.5701904296875,
 1.78778076171875,
 0.9257965087890625,
 1.329986572265625,
 1.14764404296875,
 0.9131927490234375,
 1.200714

Number of unique samples must match number of calls to propagate!

In [10]:
len(set(refs))

200

In [11]:
for ref in refs:
    print([i for i, x in enumerate(refs) if x == ref])

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[115]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[139]
[140]
[141]
[142]
[143]
[144]
[145]
[146]
[147]
[148]
[149]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]
[158]
[159]
[160]
[161]
[162]
[163]
[164]
[165]
[166]
[167]
[168]
[169]
[170]
[171]
[172]
[173]
[174]
[175]
[176]
[177]
[178]
[179]
[180]
[181]
[182]
[183]
[184]
